# Import Dependencies

In [2]:
import pandas as pd

# Read data file to dict of pandas dataframes

In [4]:
db_file = './data/question_database_schema.xlsx'
dict_of_dfs = pd.read_excel(db_file, sheet_name = None)

print(dict_of_dfs.keys())

dict_keys(['learning_objectives', 'course_objective_description', 'questions', 'answer_choices', 'exams', 'exams_learning_objectives', 'distractor_type', 'student_question_responses'])


In [6]:
print(dict_of_dfs['exams'])

   exam_id  exam_number exam_form_id
0       1A            1            A
1       1B            1            B
2       2A            2            A
3       2B            2            B
4       2C            2            C
5       3A            3            A
6       3B            3            B
7       3C            3            C
8       4A            4            A
9       4B            4            B
10      4C            4            C


# Exam descriptions

### Number of exams: 4

- Exam 1: 
    - 2 forms (A, B)
    - 16 Questions
      - 2 with 5 options
      - 14 with 4 options
- Exam 2: 3 forms (A, B, C)
    - 16 Questions
      - 1 with 3 options
      - 15 with 4 options
- Exam 3: 3 forms (A, B, C)
    - 17 Questions
      - 17 with 4 options
- Exam 4: 3 forms (A, B, C)
    - 26 Questions
      - 25 with 4 options
      - 1 with 3 options (which form of the exam do you have?)


In [22]:
completed_answer_choices = dict_of_dfs['answer_choices']
student_responses        = dict_of_dfs['student_question_responses']
#convert option_id to number indexed at 1.
if 'A' in pd.unique(completed_answer_choices['option_id']):
    completed_answer_choices['option_id'] = completed_answer_choices['option_id'] = [ ord(letter) - 64 for letter in completed_answer_choices['option_id'] ]

student_responses = student_responses[student_responses['question_id'].isin(pd.unique(completed_answer_choices['question_id']))]

student_responses_with_details = pd.merge(
    left=student_responses, 
    right=completed_answer_choices,
    how='left',
    left_on=['question_id', 'selected_option'],
    right_on=['question_id', 'option_id'],
)

student_responses_with_details


number_of_distractors_chosen = len(student_responses_with_details[student_responses_with_details['is_distractor'] < .5])
number_of_correct_answers_chosen = len(student_responses_with_details[student_responses_with_details['is_distractor'] > .5])
number_of_questions = len(student_responses_with_details)

print(f"Number of questions: {number_of_questions}.\nNumber of correct answers chosen: {number_of_correct_answers_chosen}.\nNumber of distractors chosen: {number_of_distractors_chosen}")